# Implementing definable types as systems of equations

Our goal is to make pomagma.reducer smart enough to prove that `I:UNIT`, as in
definable_types.text (2016:08:23-25) (Q2) and
operational_semantics.text (2017:01:20-02:05) (Q5.A2.9):

<b>Desired Theorem:</b> `I : A \a,a',f,x. a(f(a' x))`, where

    copy := \x,y. x y y.
    join := \x,y,z. x(y|z).
    postconj := (\r,s. <B r, B s>).
    preconj := (\r,s. <CB s, CB r>).
    compose := (\r,s,r',s'. <r o r', s' o s>).
    A = A | <I, I> | <copy, join> | <div, BOT> | <BOT, TOP> | <C, C>
          | A preconj | A postconj | A (A compose).

In [1]:
from __future__ import print_function

In [2]:
from pomagma.reducer.lib import BOT, TOP, B, C, I, pair
from pomagma.reducer.sugar import as_term
from pomagma.reducer.syntax import NVAR
from pomagma.reducer.systems import System, try_beta_step

In [4]:
i = NVAR('i')
b = NVAR('b')
c = NVAR('c')
cb = NVAR('cb')
div = NVAR('div')
copy = NVAR('copy')
join = NVAR('join')
postconj = NVAR('postconj')
preconj = NVAR('preconj')
compose = NVAR('compose')
a = NVAR('a')

system = System(
    # Basic combinators.
    i=I,
    b=B,
    c=C,
    cb=C(B),
    # Components of A.
    div=(I | as_term(lambda x: div(x, TOP))),
    copy=as_term(lambda x, y: x(y, y)),
    join=as_term(lambda x, y, z: x(y | z)),
    postconj=as_term(lambda r, s: pair(b(r), b(s))),
    preconj=as_term(lambda r, s: pair(cb(s), cb(r))),
    compose=as_term(lambda r1, s1, r2, s2: pair(b(r1, r2), b(s2, s1))),
    # Definition of A.
    a=(pair(i,i) | pair(copy, join) | pair(div, BOT) | pair(c, c) |
       a(preconj) | a(postconj) | a(a(compose))
    ),
)
print(system.pretty())

       a = (JOIN (a postconj) (JOIN (a preconj) (JOIN (a (a compose)) (JOIN (ABS (0 c c)) (JOIN (ABS (0 copy join)) (JOIN (ABS (0 div BOT)) (ABS (0 i i))))))))
       b = (ABS (ABS (ABS (2 (1 0)))))
       c = (ABS (ABS (ABS (2 0 1))))
      cb = (ABS (ABS (ABS (1 (2 0)))))
 compose = (ABS (ABS (ABS (ABS (ABS (0 (b 4 2) (b 1 3)))))))
    copy = (ABS (ABS (1 0 0)))
     div = (JOIN (ABS 0) (ABS (div 0 TOP)))
       i = (ABS 0)
    join = (ABS (ABS (ABS (2 (JOIN 0 1)))))
postconj = (ABS (ABS (ABS (0 (b 2) (b 1)))))
 preconj = (ABS (ABS (ABS (0 (cb 1) (cb 2)))))
